<a href="https://colab.research.google.com/github/KhaiTTNguyen/Computational_Quantum_Chemistry/blob/master/indole_xyz_to_gaussian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ! pip install sympy 1.3.0 
# ! pip install chempy 

ERROR: Could not find a version that satisfies the requirement 1.3.0 (from versions: none)
ERROR: No matching distribution found for 1.3.0
     |████████████████████████████████| 174kB 13.6MB/s 
     |████████████████████████████████| 92kB 9.7MB/s 
     |████████████████████████████████| 112kB 27.5MB/s 
     |████████████████████████████████| 40.6MB 109kB/s 
  Created wheel for chempy: filename=chempy-0.7.10-py2.py3-none-any.whl size=206046 sha256=71f29ba4bcaa15572447ade8c528fefeb7f7fa0d31177cf14c3b85b9de4092bd
  Stored in directory: /root/.cache/pip/wheels/39/87/ce/040d9ca0df05bcf5c4d305196a2203c6d0175c4a313a37fef3
  Created wheel for quantities: filename=quantities-0.12.4-cp36-none-any.whl size=79167 sha256=f5166004bd642e3bb213c8f4d00d8ada096a985c942bd5bc7fd45e8c9f64f8c8
  Stored in directory: /root/.cache/pip/wheels/2c/27/a5/62fb2a57d167d7b7164278de7399ba1504321c1d6c3a3802d5
  Created wheel for pyneqsys: filename=pyneqsys-0.5.7-py2.py3-none-any.whl size=27266 sha256=f461e92dcc99e900

In [4]:
# open big file
# recognize integer in front --> go to line 1000th
# start read & write new small files
#   9989 jump each time
import re, math, os,timeit
import numpy as np
import pandas as pd
import chempy as cp 


In [5]:
#get a file name to be read
def splitToMultiples(fileToRead, numGeomToSkip):
  fileReader = open(fileToRead,"r",encoding='utf-8')
  line = fileReader.readline()
  NUM_COORDS = line
  
  # skip to startLine  
  startLine = numGeomToSkip * (int(NUM_COORDS.strip()) + 2) + 1
  for _ in range(startLine):
    next(fileReader)

  # start reading
  fileCount = 0
  fileWriteName = os.path.join(".", "output","geom_"+str(fileCount)+".xyz")
  os.makedirs(os.path.dirname(fileWriteName), exist_ok=True) # create 'output' folder if not existed yet
  currFileWriter = open(fileWriteName,"a")
  
  while (line != ''):
    line = fileReader.readline()
    if re.match(' generated by VMD\n', line) : 
      continue
    elif re.match(NUM_COORDS, line):
      currFileWriter.close()
      fileCount += 1
      # creat new file --- currentFile    
      fileWriteName = os.path.join(".", "output","geom_"+str(fileCount)+".xyz")
      os.makedirs(os.path.dirname(fileWriteName), exist_ok=True) # create 'output' folder if not existed yet
      currFileWriter = open(fileWriteName,"a")
      
    else:    
      # append to new file
      currFileWriter.write(line)

  currFileWriter.close()
  fileReader.close()


fileToRead = "UC-MM-PBC.xyz"
splitToMultiples(fileToRead, 1000)

In [ ]:
# mydir = "output"
# for f in os.listdir(mydir):
#     if not f.endswith(".xyz"):
#         continue
#     os.remove(os.path.join(mydir, f))

In [6]:
# configurations
# this script is applicatble only for ring-molecules
RADIUS = 6.00 # angstrom -- since CoM in a couple of angstrom inside the INDOLE molecule
INDOLE_NUM_ATOM = 16
# change "header" in genGaussianInput(fileName) if use different molecule

In [20]:
def genGaussianInput(fileName):
  '''
    take in file
    store 
    filter
      find CoM_indole
      keep looping till end of file / 3 lines at a time 
        True: write 3 lines
    close
  '''
  filePath = os.path.join(".", "output",fileName)

  data = pd.read_csv(filePath, sep="\\s+", header=None)
  data[0] = data[0].apply(lambda x: cp.util.periodic.mass_from_composition(cp.util.parsing.formula_to_composition(x[0])))


  np_mass, np_x, np_y, np_z = data[0].to_numpy(dtype=np.float64), \
                                  data[1].to_numpy(dtype=np.float64), \
                                  data[2].to_numpy(dtype=np.float64), \
                                  data[3].to_numpy(dtype=np.float64)      
  #----------------------------------------------------
  start_indx = INDOLE_NUM_ATOM  
  indole_CoM = np.asarray([np.dot(np_mass[:start_indx],np_x[:start_indx])/np.sum(np_mass[:start_indx]), \
                           np.dot(np_mass[:start_indx],np_y[:start_indx])/np.sum(np_mass[:start_indx]), \
                           np.dot(np_mass[:start_indx],np_z[:start_indx])/np.sum(np_mass[:start_indx])])


  fileReader = open(filePath,"r",encoding='utf-8')

  # -------------------- start writing --------------------
  fileWriteName = os.path.join(".", "GaussInput",os.path.splitext(fileName)[0]+".inp")
  os.makedirs(os.path.dirname(fileWriteName), exist_ok=True) # create folder if not existed yet
  fileWriter = open(fileWriteName,"w+")

  # write header
  header = '{}\n{}\n{}\n\n{}\n\n{}\n'.format('%mem=1gb', 
                                                    '%nproc=4',
                                                    '#P TD(NStates=5) CAM-B3LYP/6-31G(d) SP',
                                                    'C8H7N-excited',
                                                    '0 1')      
  fileWriter.write(header)

  # read INDOLE over
  for _ in range(start_indx):
    line = fileReader.readline()
    first = line.split()[0]
    line = line.replace(first,first[0])
    fileWriter.write(line)

  # read WATERs over
  while start_indx < len(np_mass):
    '''
    compute CoM
    satisfy, write lines
    '''
    water_CoM = np.asarray([np.dot(np_mass[start_indx:start_indx+3],np_x[start_indx:start_indx+3])/np.sum(np_mass[start_indx:start_indx+3]), \
                            np.dot(np_mass[start_indx:start_indx+3],np_y[start_indx:start_indx+3])/np.sum(np_mass[start_indx:start_indx+3]), \
                            np.dot(np_mass[start_indx:start_indx+3],np_z[start_indx:start_indx+3])/np.sum(np_mass[start_indx:start_indx+3])])
    # ---- filter-----
    numline = 0
    if np.linalg.norm(indole_CoM - water_CoM) <= RADIUS:
      while numline < 3:
        line = fileReader.readline()
        first = line.split()[0]
        line = line.replace(first,first[0])
        fileWriter.write(line)
        numline=numline+1  
    else:
      while numline < 3:
        line = fileReader.readline()
        numline=numline+1
    start_indx = start_indx + 3

  fileWriter.close()
  fileReader.close()

In [21]:
# can use multiprocessing for this
mydir = "output"
for f in os.listdir(mydir):
    genGaussianInput(f)

In [19]:
mydir = "GaussInput"
for f in os.listdir(mydir):
    if not f.endswith(".inp"):
        continue
    os.remove(os.path.join(mydir, f))
# from google.colab import drive
# drive.mount('/content/drive')

In [11]:
 !zip -r GaussInput.zip GaussInput/ 

updating: GaussInput/ (stored 0%)


In [ ]:
cp.util.periodic.mass_from_composition(cp.util.parsing.formula_to_composition('C'))


12.011

In [ ]:
fileWriteName = os.path.join(".", "GaussInput",os.path.splitext(fileName)[0]+".inp")
fileWriteName

'./GaussInput/geom_10.inp'

In [ ]:
line1 = 'OW        23.130001       -1.129798       30.950003'
line2 = 'HW1       23.650002       -1.919798       30.630003'
first = line2.split()[0]
line2 = line2.replace(first,first[0])
line2

'H       23.650002       -1.919798       30.630003'

In [ ]:
# first = line2.split()[0]
#         line = line.replace(first,first[0])

In [ ]:
s = "hello"
print(s.capitalize())  # Capitalize a string; prints "Hello"
print(s.upper())       # Convert a string to uppercase; prints "HELLO"
print(s.rjust(7))      # Right-justify a string, padding with spaces; prints "  hello"
print(s.center(7))     # Center a string, padding with spaces; prints " hello "
print(s.replace('l', '(ell)'))  # Replace all instances of one substring with another;
                                # prints "he(ell)(ell)o"
print('  world '.strip())  # Strip leading and trailing whitespace; prints "world"

In [ ]:
'''
files are splited and jobs are executed in order
--> CAN'T USE: results must be sequential

Credit: https://www.blopig.com/blog/2016/08/processing-large-files-using-python/
'''

# import multiprocessing as mp,os

# FILENAME = "spark-sample.txt"
# def process_wrapper(chunkStart, chunkSize):
#     with open(FILENAME) as f:
#         f.seek(chunkStart)
#         lines = f.read(chunkSize).splitlines()
#         for line in lines:
#             print(line)

# def chunkify(fname,size=10):
#     fileEnd = os.path.getsize(fname)
#     with open(fname,'rb+') as f:
#       chunkEnd = f.tell() # start of file
#       while True:
#           chunkStart = chunkEnd
#           f.seek(size,1) # ---> 1 (seek relative to the current position)
#           f.readline()
#           chunkEnd = f.tell()
#           yield chunkStart, chunkEnd - chunkStart
#           if chunkEnd >= fileEnd:
#               break

# #init objects
# pool = mp.Pool(4)
# jobs = []

# #create jobs
# for chunkStart,chunkSize in chunkify(FILENAME):
#     jobs.append( pool.apply_async(process_wrapper,(chunkStart,chunkSize)) )

# #wait for all jobs to finish
# for job in jobs:
#     job.get()

# #clean up
# pool.close()

In [39]:
def addEndline(fileName):
  '''
    take in file
    seek(end)
    add newline
    close
  '''
  f = open(fileName, 'a+')
  f.write("\n")
  f.close()

In [40]:
# can use multiprocessing for this
def computeGauss():
  mydir = "GaussInput"
  for f in os.listdir(mydir):
    addEndline(f)

computeGauss()